# 1. Install and Import Dependencies

In [1]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [2]:
#%pip install stable-baselines3[extra] protobuf==3.20.*

In [3]:
#%pip install mss pydirectinput pytesseract

In [4]:

#%pip install gymnasium

In [5]:
import os
from mss import mss
import pydirectinput
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import time
#import gymnasium as gym
#from gymnasium import spaces
import gym
from gym import spaces
from TerrarianEyes import TerrarianEyes
%matplotlib inline

# 2. Build the Environment

## 2.1 Create Environment

In [6]:
%reload_ext autoreload
%autoreload 2
class TerrEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}
    def __init__(self):
        super().__init__()
        # Setup spaces
        self.observation_space = spaces.Dict(
            {
                'map' :       spaces.Box(low=0, high=11, shape=(67, 120), dtype=np.int8),
                'inventory' : spaces.Box(low=0, high=11, shape=(9,10), dtype=np.int8),
            }
        )
        self.action_space = spaces.Discrete(7)
        # Capture game frames
        self.cap = mss()
        self.game_location = {'top': 0, 'left': 0, 'width': 1920, 'height': 1080}
        self.done_location = {'top': 460, 'left': 760, 'width': 400, 'height': 70}
        self.action_map = {
            0: 'no_op',
            1: 'space',
            2: 'd', 
            3: 'a', 
            4: 'attack',
            5: 'cut'
        }
        # Create Instance
        tiles_weights_path = os.path.join('runs', 'train', 'yolov5s6-tiles', 'weights', 'best.pt')
        objects_weights_path = os.path.join('runs', 'train', 'yolov5l6-objects', 'weights', 'best.pt')
        self.eyes = TerrarianEyes(tiles_weights_path, objects_weights_path)
        self.timer = None
        self.time_limit = 360
        self.day_timer = time.time()
        self.day_limit = 300
           
    def step(self, action):
        if self.timer is None:
            raise AssertionError("Cannot call env.step() before calling reset()")
        # Get current health
        health = self.eyes.map.getHealth()
        reward = 0
        if action == 1:
            # press 'action' key and hold it down
            pydirectinput.keyDown(self.action_map[action])

            # keep the key pressed for 2 seconds
            time.sleep(0.3)
            # release the 'action' key
            pydirectinput.keyUp(self.action_map[action])
        elif action < 4:
            # press 'action' key and hold it down
            pydirectinput.keyDown(self.action_map[action])

            # keep the key pressed for 2 seconds
            time.sleep(0.3)
            # release the 'action' key
            pydirectinput.keyUp(self.action_map[action])
            reward = 1 
        else: 
            # In case we need map or inventory
            if action == 4: # attack
                # find closest enemy position and check 
                #with open("delete.txt", 'w') as f:
                #    f.write(str(self.eyes.map))
                attack, x, y= self.eyes.map.isEnemyOnAttackRange()
                # if is in attack range
                if attack:
                    # Move mouse to enemy position
                    pydirectinput.moveTo((x+1)*16 + 16, y*16 + 8)
                    # attack
                    pydirectinput.press('3')
                    # Press the left mouse button
                    pydirectinput.mouseDown(button='left')
                    time.sleep(6)

                    # Release the left mouse button
                    pydirectinput.mouseUp(button='left')
                    reward = 20
            elif action == 5: # cut wood
                # find closest tree position and check 
                # if is in cut range
                cut, x, y = self.eyes.map.isTreeOnCutRange()
                # if is in attack range
                if cut:
                    # Move mouse to enemy position
                    pydirectinput.moveTo((x+1)*16 + 16, y*16 + 8)
                    
                    # attack
                    pydirectinput.press('3')
                    # Press the left mouse button
                    pydirectinput.mouseDown(button='left')                    
                    time.sleep(2)
                    # Release the left mouse button
                    pydirectinput.mouseUp(button='left')
                    reward = 10

        done, _ = self.get_done() 
        observation = self.get_observation()
        new_health = self.eyes.map.getHealth()
        if new_health - health < 0 or done:
            reward = reward - 20
        # calculate real reward
        info = {}
        return observation, reward, done, info
        
    
    def reset(self):
        time.sleep(10)
        pydirectinput.click(x=930, y=512)
        self.timer = time.time()
        observation = self._get_obs()
        return observation
        
    def render(self):
        cv2.imshow('Game', self.current_frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.close()
         
    def close(self):
        cv2.destroyAllWindows()
    
    #def _get_obs(self):
        #return {"map": self.eyes.map.current_map, "inventory": self.eyes.inventory.inventory}

    def _get_obs(self):
        return {"map": self.eyes.map.current_map, "inventory": self.eyes.inventory.inventory}

    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        self.eyes.updateMap(raw)
        self.eyes.updateInventory(raw)
        #gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        #resized = cv2.resize(gray, (1920,1080))
        #channel = np.reshape(resized, (1,1080,1920))
        return {"map": self.eyes.map.current_map, "inventory": self.eyes.inventory.inventory}
    
    def get_done(self):
        done = False
        done_cap = None                   

        #elif time.time() - self.timer  > self.time_limit:
        #    done = True
        done_cap = np.array(self.cap.grab(self.done_location))
        done_strings = ['You', 'You ']
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        if time.time() - self.day_timer  > self.day_limit:
            # reset day

            # Open inventory
            pydirectinput.press('Esc')
            # Click Power menu
            pydirectinput.moveTo(50, 315)
            # Press the left mouse button
            pydirectinput.mouseDown(button='left')
            # Release the left mouse button
            pydirectinput.mouseUp(button='left')

            #Click other place in the power menu to reset view
            pydirectinput.moveTo(50, 530)
            # Press the left mouse button
            pydirectinput.mouseDown(button='left')
            # Release the left mouse button
            pydirectinput.mouseUp(button='left')

            #Click time menu
            pydirectinput.moveTo(50, 630)
            # Press the left mouse button
            pydirectinput.mouseDown(button='left')
            # Release the left mouse button
            pydirectinput.mouseUp(button='left')

            #Click dawn
            pydirectinput.moveTo(115, 655)
            # Press the left mouse button
            pydirectinput.mouseDown(button='left')
            # Release the left mouse button
            pydirectinput.mouseUp(button='left')

            
            # Close inventory
            pydirectinput.press('Esc')

            self.day_timer = time.time() 
        return done, done_cap

# 2.2 Test Environment

In [7]:
env = TerrEnv()

YOLOv5  2023-3-26 Python-3.9.2rc1 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model summary: 206 layers, 12319756 parameters, 0 gradients, 16.1 GFLOPs
Fusing layers... 
Model summary: 346 layers, 76157124 parameters, 0 gradients, 110.0 GFLOPs


In [8]:
#obs=env.get_observation()
#env.eyes.updateMap(env.observation)
#env.eyes.map.getHealth()

In [9]:
#plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))

In [10]:
#done, done_cap = env.get_done()

In [11]:
#plt.imshow(done_cap)

In [12]:
#pytesseract.image_to_string(done_cap)[:4]

In [13]:
#for episode in range(10): 
#    obs = env.reset()
#    done = False
#    total_reward = 0
#    while not done:
#        action = env.action_space.sample()
#        obs, reward,  done, info =  env.step(env.action_space.sample())
#        total_reward  += reward
#    print('Total Reward for episode {} is {}'.format(episode, total_reward))    

# 3. Train the Model

## 3.1 Create Callback

In [14]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker
from gym.utils.env_checker import check_env

In [15]:
#env_checker.check_env(env)
#check_env(env)

In [16]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [17]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [18]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

## 3.2 Build DQN and Train

In [19]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [20]:
env = TerrEnv()

YOLOv5  2023-3-26 Python-3.9.2rc1 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)

Fusing layers... 
Model summary: 206 layers, 12319756 parameters, 0 gradients, 16.1 GFLOPs
Fusing layers... 
Model summary: 346 layers, 76157124 parameters, 0 gradients, 110.0 GFLOPs


In [21]:
model = DQN('MultiInputPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=1200000, learning_starts=10)
model.load('train/best_model_7000') 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
time1 = time.time()
model.learn(total_timesteps=3001, callback =callback)
time2 = time.time()
print(f'time for 3000 step: {str(time2 - time1)}')

Logging to ./logs/DQN_15
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 289      |
|    ep_rew_mean      | -179     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 0        |
|    time_elapsed     | 1648     |
|    total_timesteps  | 1155     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.82     |
|    n_updates        | 286      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 182      |
|    ep_rew_mean      | -79.1    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 0        |
|    time_elapsed     | 2125     |
|    total_timesteps  | 1456     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.53  

In [ ]:
model.load('train_first/best_mode l_50000') 

# 4. Test out Model

In [ ]:
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, findOut = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)